In [1]:
from ollama import Client
c = Client(host="http://13.212.147.243:11434")
resp = c.chat(model="llama3.1:8b",
              messages=[{"role":"user","content":"Hello"}],
              stream=False,
              options={"num_ctx": 2048})
print(resp)

model='llama3.1:8b' created_at='2025-08-11T11:12:26.8707353Z' done=True done_reason='stop' total_duration=15370733300 load_duration=3936577500 prompt_eval_count=11 prompt_eval_duration=2551976700 eval_count=23 eval_duration=8880623400 message=Message(role='assistant', content="It's nice to meet you. Is there something I can help you with or would you like to chat?", thinking=None, images=None, tool_name=None, tool_calls=None)


In [4]:
from ollama import Client

def sizeof(n):
    if n is None: return "-"
    for u in ("B","KB","MB","GB","TB"):
        if n < 1024: return f"{n:.1f} {u}"
        n /= 1024
    return f"{n:.1f} PB"

def val(x, default="-"):  # normalize None/empty
    return default if x in (None, "", {}) else x

c = Client(host="http://13.212.147.243:11434")
tags = c.list()
models = getattr(tags, "models", tags.get("models", []))

for m in models:
    name   = val(getattr(m, "name", m.get("name")))
    size   = getattr(m, "size", m.get("size"))
    det    = getattr(m, "details", m.get("details", {})) or {}
    params = val(getattr(det, "parameter_size", det.get("parameter_size")))
    quant  = val(getattr(det, "quantization_level", det.get("quantization_level")))
    print(f"{name:40} params={str(params):>8}  quant={str(quant):>10}  size={sizeof(size)}")


-                                        params=    8.0B  quant=    Q4_K_M  size=4.6 GB


In [9]:
from ollama import Client

c = Client(host="http://13.212.147.243:11434")
info = c.show(model="llama3.1:8b")  # hits POST /api/show
print(info)  # includes full template, parameters, license, etc.


modified_at=datetime.datetime(2025, 8, 7, 18, 22, 43, 422694, tzinfo=TzInfo(UTC)) template='{{- if or .System .Tools }}<|start_header_id|>system<|end_header_id|>\n{{- if .System }}\n\n{{ .System }}\n{{- end }}\n{{- if .Tools }}\n\nCutting Knowledge Date: December 2023\n\nWhen you receive a tool call response, use the output to format an answer to the orginal user question.\n\nYou are a helpful assistant with tool calling capabilities.\n{{- end }}<|eot_id|>\n{{- end }}\n{{- range $i, $_ := .Messages }}\n{{- $last := eq (len (slice $.Messages $i)) 1 }}\n{{- if eq .Role "user" }}<|start_header_id|>user<|end_header_id|>\n{{- if and $.Tools $last }}\n\nGiven the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.\n\nRespond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.\n\n{{ range $.Tools }}\n{{- . }}\n{{ end }}\nQuestion: {{ .Content }}<|eot_

In [11]:
from ollama import Client

c = Client(host="http://13.212.147.243:11434")
procs = c.ps()  # hits GET /api/ps
for p in (procs.models if hasattr(procs, "models") else procs.get("models", [])):
    # fields typically include: name, size, digest
    print(p.get("name"), p.get("size"))


In [12]:
import requests, pprint
host = "http://13.212.147.243:11434"

# list models
r = requests.get(f"{host}/api/tags", timeout=30)
pprint.pp(r.json())

# details for one model
r = requests.post(f"{host}/api/show", json={"name": "llama3.1:8b"}, timeout=30)
pprint.pp(r.json())

# loaded models (processes)
r = requests.get(f"{host}/api/ps", timeout=30)
pprint.pp(r.json())


{'models': [{'name': 'llama3.1:8b',
             'model': 'llama3.1:8b',
             'modified_at': '2025-08-07T18:22:43.4226944Z',
             'size': 4920753328,
             'digest': '46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e',
             'details': {'parent_model': '',
                         'format': 'gguf',
                         'family': 'llama',
                         'families': ['llama'],
                         'parameter_size': '8.0B',
                         'quantization_level': 'Q4_K_M'}}]}
{'license': 'LLAMA 3.1 COMMUNITY LICENSE AGREEMENT\n'
            'Llama 3.1 Version Release Date: July 23, 2024\n'
            '\n'
            '“Agreement” means the terms and conditions for use, reproduction, '
            'distribution and modification of the\n'
            'Llama Materials set forth herein.\n'
            '\n'
            '“Documentation” means the specifications, manuals and '
            'documentation accompanying Llama 3.1